# 1D Heat Equation with Forward Euler and No‑Flux (Neumann) Boundary Conditions

We solve the 1D heat equation on $x\in[0,L]$:

$$
u_t = \alpha u_{xx}
$$

with **no‑flux (insulated) boundary conditions**:

$$u_x(0,t)=0, \quad u_x(L,t)=0.$$

We build the method gradually:

1. Discretise space with a grid.
2. Approximate $u_{xx}$ with second differences.
3. Apply **Forward Euler** in time.
4. Implement the Neumann boundary conditions.
5. Visualise the solution (snapshots + animation).


## 0. Imports
We use NumPy for arrays and Matplotlib for plotting.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# For animations in notebooks
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


## 1. Grid and parameters

Let $x_i = i\,\Delta x$ for $i=0,1,\dots,N$.
We store the solution values $u_i(t) \approx u(x_i,t)$ in a NumPy array.

We also choose a time step $\Delta t$.

### Stability (important)

Forward Euler for diffusion is conditionally stable. A standard stability condition is:

$$
\Delta t \le \frac{\Delta x^2}{2\alpha}.
$$

We will compute `dt` from a chosen safety factor.

In [2]:
# Physical / model parameters
L = 1.0          # Domain length
alpha = 1.0      # Diffusion coefficient

# Spatial grid
N = 200          # Number of intervals (so N+1 grid points)
x = np.linspace(0.0, L, N + 1)
dx = x[1] - x[0]

# Time step chosen from stability limit
safety = 0.05  # <= 1.0, smaller is safer
dt_stable = dx**2 / (2 * alpha)
dt = safety * dt_stable

dx, dt, dt_stable

(0.005, 6.25e-07, 1.25e-05)

## 2. Initial condition

We pick an initial temperature profile. Two common choices:

- A Gaussian bump
- A piecewise constant "hot spot"

You can change the initial condition cell and rerun.

In [ ]:
def initial_condition(x):
    # Gaussian bump centred at L/2
    return np.exp(-1000 * (x - 0.5 * L)**2)

u0 = initial_condition(x)

plt.figure()
plt.plot(x, u0)
plt.xlabel('x')
plt.ylabel('u(x,0)')
plt.title('Initial condition')
plt.show()

## 3. Discretising $u_{xx}$ (interior points)

For interior points $i=1,\dots,N-1$ we use the second difference:

$$
u_{xx}(x_i,t) \approx \frac{u_{i-1}-2u_i+u_{i+1}}{\Delta x^2}.
$$

Forward Euler in time gives:

$$
u_i^{n+1} = u_i^n + \alpha\,\Delta t\,\frac{u_{i-1}^n-2u_i^n+u_{i+1}^n}{\Delta x^2}.
$$

Let
$$r = \alpha\,\Delta t / \Delta x^2.$$
Then the update is:
$$u_i^{n+1} = u_i^n + r\,(u_{i-1}^n-2u_i^n+u_{i+1}^n).$$

In [18]:
r = alpha * dt / dx**2
r

0.025

## 4. No‑flux (Neumann) boundary conditions

No flux means $u_x=0$ at the boundaries.

A simple finite-difference way to enforce this is to use **ghost points**:

- At $x=0$: $u_x(0,t)\approx (u_1-u_{-1})/(2\Delta x)=0 \Rightarrow u_{-1}=u_1$.
- At $x=L$: $u_x(L,t)\approx (u_{N+1}-u_{N-1})/(2\Delta x)=0 \Rightarrow u_{N+1}=u_{N-1}$.

Substituting these into the second difference at the boundaries gives:

- At $i=0$:
$$
u_{xx}(x_0,t) \approx \frac{u_{-1}-2u_0+u_1}{\Delta x^2} = \frac{2(u_1-u_0)}{\Delta x^2}.
$$
- At $i=N$:
$$
u_{xx}(x_N,t) \approx \frac{u_{N-1}-2u_N+u_{N+1}}{\Delta x^2} = \frac{2(u_{N-1}-u_N)}{\Delta x^2}.
$$

We’ll implement these two special updates for the endpoints, and use the standard formula in the interior.

## 5. One time step function

We write a function that maps the current array `u` to the next array `u_next`.

In [19]:
def step_forward_euler_neumann(u, r):
    """One Forward Euler step for u_t = alpha u_xx with zero-flux boundaries.

    Parameters
    ----------
    u : ndarray, shape (N+1,)
        Current solution values on grid points.
    r : float
        r = alpha*dt/dx^2.

    Returns
    -------
    u_next : ndarray
        Solution after one time step.
    """
    u_next = u.copy()
    N = len(u) - 1

    # Interior update: i = 1..N-1
    u_next[1:N] = u[1:N] + r * (u[0:N-1] - 2*u[1:N] + u[2:N+1])

    # Neumann boundaries via ghost points: u_{-1}=u_1, u_{N+1}=u_{N-1}
    u_next[0] = u[0] + r * (2*(u[1] - u[0]))
    u_next[N] = u[N] + r * (2*(u[N-1] - u[N]))

    return u_next


Quick sanity check: apply a few steps and plot. The solution should smooth out over time.

In [ ]:
u = u0.copy()
steps = 200
for _ in range(steps):
    u = step_forward_euler_neumann(u, r)

plt.figure()
plt.plot(x, u0, label='t=0')
plt.plot(x, u, label=f't={steps*dt:.4f}')
plt.xlabel('x')
plt.ylabel('u')
plt.title('Smoothing by diffusion (no-flux boundaries)')
plt.legend()
plt.show()

## 6. Full simulation and snapshots

We run the method for a chosen final time and store a few snapshots for plotting.

In [ ]:
T = 0.05                         # Final time
num_steps = int(np.ceil(T / dt)) # Number of time steps

# Choose snapshot times (in steps)
snapshot_fracs = [0.0, 0.1, 0.25, 0.5, 1.0]
snapshot_steps = sorted({int(f * num_steps) for f in snapshot_fracs})

u = u0.copy()
snapshots = {0: u.copy()}

for n in range(1, num_steps + 1):
    u = step_forward_euler_neumann(u, r)
    if n in snapshot_steps:
        snapshots[n] = u.copy()

plt.figure()
for n in snapshot_steps:
    t = n * dt
    plt.plot(x, snapshots[n], label=f't={t:.4f}')
plt.xlabel('x')
plt.ylabel('u')
plt.title('Snapshots of the Forward Euler solution')
plt.legend()
plt.show()

## 7. Animation

This animates the solution over time.

If the animation does not display in your environment, you can still use the snapshot plots above.

In [ ]:
# Re-run simulation but keep frames for animation
T_anim = 0.05
num_steps_anim = int(np.ceil(T_anim / dt))
frame_stride = max(1, num_steps_anim // 200)  

u = u0.copy()
frames = [u.copy()]
times = [0.0]

for n in range(1, num_steps_anim + 1):
    u = step_forward_euler_neumann(u, r)
    if n % frame_stride == 0:
        frames.append(u.copy())
        times.append(n * dt)

fig, ax = plt.subplots()
line, = ax.plot(x, frames[0])
ax.set_xlabel('x')
ax.set_ylabel('u')
title = ax.set_title(f't = {times[0]:.4f}')
ax.set_ylim(min(u0.min(), frames[-1].min()) - 0.05, max(u0.max(), frames[-1].max()) + 0.05)

def update(i):
    line.set_ydata(frames[i])
    title.set_text(f't = {times[i]:.4f}')
    return line, title

anim = FuncAnimation(fig, update, frames=len(frames), interval=50, blit=False)
plt.close(fig)

HTML(anim.to_jshtml())

## 8. Demonstrate instability

If you choose a time step that violates the stability condition, Forward Euler can blow up.

Try setting `safety = 1.2` (so that `dt > dx^2/(2alpha)`) and rerun from the parameter cell.
You should see oscillations and rapidly growing values.
